<a href="https://colab.research.google.com/github/AguaClara/CEE4520/blob/master/Unit_Process_Designs/Entrance_tank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrance Tank: Executive Summary
## 29 June 2020 
The trash rack and plate settlers in the entrance tank at Gracias have been unsuccessful at capturing grit and preventing it from traveling to the rest of the plant. This grit causes maintenance issues by settling in the flocculator and the channel leading to the sedimentation tanks. To solve this problem, the plate settlers could be replaced with a more traditional horizontal settling basin. A series of 4-sided hoppers fitted with a drainpipe would make it easy to remove grit from the bottom of the chamber via gravity. However, Kawamura advises against the use of square tanks, due to their poor flow patterns:

> “The square tank is satisfactory for the purpose of wastewater treatment but does not perform well as a water treatment grit chamber. The main problem with the detritus [square] tank is a function of its physical characteristics. The short tank length and the uneven inflow to the tank cause severe flow short-circuiting. … To provide good flow characteristics for grit settling, the chamber should be rectangular with a contracted inlet (pg. 414).” 

**To settle particles greater than 0.1 mm in diameter, the entrance tank at Gracias would need to be 11.13 m in length.** This calculation was done under the assumption of a 60 L/s flow rate, and that the entrance tank is the same width as the flocculator.  According to Kawamura, sand and silt smaller than 0.1 mm do not pose a significant risk to downstream processes (pg. 416). For context, a single treatment train (excluding the filters) at Gracias has an approximate 15 m x 7 m area. Increasing the area of the entrance tank would drive up construction costs and may block plant operator access to important parts of the plant. For example, observing the flocs in the first 4 baffles of the flocculator is one of the most reliable indications of plant performance. 

The required plan-view area of the entrance tank could be reduced by increasing the critical particle size that needs to be captured. Kawamura recommended a 0.1 mm diameter based on pump and piping damage in downstream processes. However, the predominant issue in Gracias, Honduras is grit settling in the flocculator and inlet channel to the sedimentation tanks.

The velocities going around the bottom bend of the flocculator are comparatively higher than velocities in the entrance tank. This suggests that particles larger than 0.1 mm could be scoured from the bottom of the flocculator, and that the entrance tank may be designed for a larger critical particle diameter. **The shear on the bottom of the flocculator should be calculated to determine the largest particle that can be scoured.**

Wave action and splashing is another issue occuring in the entrance tank at the plant in Gracias, Honduras. The velocity and kinetic energy was calculated at various points within the entrance tank (Table 1): exiting the influent pipe, through the plate settlers, and at the shallow channel leading to the LFOM. 

*Table 1: The velocity and kinetic energy at various location in the entrance tank*

|Location|Velocity (m/s)|Kinetic Energy (cm)|
|:---:|:---:|:---:|
|Influent Pipe|1.91|18.6|
|Plate Settlers|0.16|0.1305|
|LFOM Channel|0.1593|0.1294|

The velocity exiting the influent pipe is much greater than other areas of the entrance tank. **This suggests that the kinetic energy exiting the influent pipe may be the primary source of wave action.** Various methods should be explored to reduce this kinetic energy. This design may include the following components:
- aiming the influent pipe at the end wall of the entrance tank
- use 2" pipes as diffusers
- placing obstacles along the side walls of entrance tank
- including a diffuser wall (the trash racks may assist with this)
- place an endcap on the influent pipe so that water does not flow straight down into the hopper, and cut a vertical slit down the side to act as a diffuser. 

**Each component should be evaluated on its effectiveness at dissipating kinetic energy and its ease of construction.** 


# Python Calculations

In [1]:
!pip install aguaclara
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import numpy as np

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 204kB 8.2MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.3-cp36-none-any.whl size=99459 sha256=74f3b734412ab742a3418ee23494523da3b0c4866d53181a1ed6413ece9a668c
  Stored in directory: /root/.cache/pip/wheels/da/f3/b6/a4648b3b3bc239e7953e74a2dee9f21c07d1b1e1f65cb86311
Successfully built aguaclara


## Calculate the largset particle that will be scoured by the inlet channel to the sedimentation tanks: velocity approach


In [10]:
# Calculate the smallest particle that will be scoured from in the inlet channel to the sedimentation tanks: scour velocity approach
# assume that the velocity of water in the channel is relatively constant due to a sloped bottom geometry
q = (60 * u.L/u.s).to(u.m**3/u.s)
T = 20 * u.degC
rho_H20 = ac.density_water(T)
rho_p = 2650 *u.kg/u.m**3 # density of sand
theta = (4.8*u.deg).to(u.rad)
w_chan = 0.5 * u.m
h_chan = 0.156 * u.m # the height of the channel at the influent end
a_chan = w_chan*(h_chan - 0.1 * u.m) # The inlet channel to the sedimentation tanks have a 10 cm freeboard
v_chan = q/a_chan

d = (3*0.332*rho_H20*v_chan**2)/(2*500000**0.5*(rho_p - rho_H20)*ac.GRAVITY*np.tan(theta))

print('The velocity in the influent channel to the sedimentation tank is ', v_chan)
print('The largest diameter particle that will be scoured by the inlet channel to the sed tank is ', d.to(u.mm))

# Calculate the require length of the entrance tank assuming it is the same width as the flocculator at Gracias

nu = ac.viscosity_kinematic_water(T)
w = 0.60 * u.m # assume ET is same width as the flocculator
v_c = ((rho_p - rho_H20)*ac.GRAVITY*d**2)/(18*nu*rho_H20)
print('The cricitcal velocity is ',v_c.to(u.mm/u.s))

# check that flow is in laminar regime for Stokes' law to be valid
Re = v_c*d/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area = q/v_c
length_ET = area/w
print('The entrance tank length is ', length_ET)

# repeat calculation in transistion regime
mu = nu*rho_H20 # dynamic viscosity of water
print('The calculation needs to repeated using Stokes Law for transition flow')
v_c_transition = (((rho_p - rho_H20)*ac.GRAVITY*d**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4)

# check that flow is in the transition/turbulent regime
Re = v_c*d/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area_2 = q/v_c_transition
length_ET_2 = area_2/w
print('The entrance tank length is ', length_ET_2)

The velocity in the influent channel to the sedimentation tank is  2.143 meter / second
The largest diameter particle that will be scoured by the inlet channel to the sed tank is  2.373 millimeter
The cricitcal velocity is  5060 millimeter / second
The Reynolds number is  1.196e+04 dimensionless
The entrance tank length is  0.01976 meter
The calculation needs to repeated using Stokes Law for transition flow
The Reynolds number is  1.196e+04 dimensionless
The entrance tank length is  0.2403 meter ** 1


## Calculate the minimum particle size that will be scoured from the bottom of the flocculator

**Sources** (still need to site these properly)
1. [Hydraulic Engineering: Channel Design Powerpoint](https://github.com/monroews/Hydraulics/raw/master/10_Channel_Design.pptx) (slides 19-23)
1. Chapter 12: Stable Channel Design Functions in [HEC-RAS River Analysis System Hydraulic Reference Manual](https://www.hec.usace.army.mil/software/hec-ras/documentation/HEC-RAS%205.0%20Reference%20Manual.pdf)  (page 12-21)
1. Chapter 9 [Introduction to Fluid Motion, Sediment Transport and Current Generated Sedimentary Structures](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf)

**Strategy:**
   1. Find the shear $\tau_o$ on the bottom of the flocculator
   1. Use the updated Shields diagram (Figure 1) to identify the largest grain size that will be scoured by the flocculator shear stress

**Summary of Relevant Literature**


Shear in the flocculator:
- If the floor of the flocculator is flat, velocity gradients would be higher in the downstream end of the flocculator, due to lower water levels. Therefore, we are most interested in know the velocity gradients/shear on the upstream end of the flocculator.
- at lower flow rates, velocity gradients will also be lower. We will assume that the plant is pretty much always running at full capacity.



---

The movement of particles can be described by 3 dimensionless parameters: (source 3, pg. 268)

**1. Density ratio:** The density ratio is only suspected to have a small effect on the initiation of particle movement, and there is little literature on it (source 3, pg. 281). It will not be used in calculations here. 
$$\frac{\rho_s}{\rho}$$

where

$\rho_s$ = the density of the particle

$\rho$ = the density of the fluid 

<br/>

**2. 'Shear' or 'Boundary' Reynold's Number:**
$$Re_* = \frac{u_*D}{\nu}$$

where

$u_*$ = shear velocity

D = particle diameter

$\nu$ = kinematic viscosity

<br/>

**Shear Velocity:** (source 1)
$$u_* = \sqrt{\frac{\tau_o}{\rho}} = \sqrt{gR_hS_f}$$ 

where

$g$ = acceleration due to gravity

$R_h$ = hydraulic radius (ie. cross-sectional area of the flow divided by the wetted perimeter)

$S_f$ = slope??????

<br/>

**3. Shield's Parameter:** 

$$\frac{\tau_o}{\Delta\rho gD} = \frac{2}{3}tan\theta$$

According to source 3, the fraction on the righthand side appears to depend on geometrical and dynamical effects (pg. 267). Also note that multiplying the numerator and denominator by $D^2$ gives a ratio of fluid force on the particle to the weight of the particle (pg. 269).

In Shield's experiment, "Bed shear stress was determined from the resistance equation $\tau_o = \gamma dsin\phi"$ (source 3, pg. 273)

**The Shields diagram can be 'recast' so that only one axis contains the variable $\tau_o$ and only the other axis contains the particle diameter D.** These new axis are as follows (source 3, pg. 275):

$$\tau_o(\rho /(g\Delta\rho)^2\mu^2)^{1/3}$$

$$D(\rho (g\Delta\rho)/\mu^2)^{1/3}$$

Source 3 provides an updated version of Shields diagram reframed in terms of shear velocity and particle diameter at $20 ^\circ C$ (pg. 277):

![Shields Diagram](https://github.com/Emily-Wood/AguaClara/blob/master/Updated%20Shields%20Diagram.PNG?raw=true)

**Figure 1**: an updated Shields diagram in terms of shear velocity and particle diameter at $20 ^\circ C$ (source 3, page 277)






In [4]:
# Calculate the shear on the bottom of the flocculator
G =  40 / u.s # velocity gradient (this number is just a test, going to change it later)
T = 20 * u.degC
tau = ac.viscosity_dynamic_water(T) * G
rho = ac.density_water(T)
U_star = (tau/rho)**0.5

print(U_star.to(u.cm/u.s))

# this gives very roughy 0.2 mm with a good bit of rounding. 


#################
# calculating the velocity in the inlet channel between the flocculator and sed tank:
hl_sed = 5 *u.cm  #headloss in the sed tank from Monroe
SED_TANK_Q_RATIO = 0.95   #known flow ratio for sed tank
v_chan = (np.sqrt(hl_sed*2*ac.GRAVITY/(1-SED_TANK_Q_RATIO**2))).to(u.m/u.s)
print(v_chan)


0.6336 centimeter / second
3.171 meter / second


## Calculate the kinetic energy and velocity exiting the influent pipe at Gracias


In [4]:
q = (60 * u.L/u.s).to(u.m**3/u.s)
ID = 0.2 * u.m # inner diameter of influent pipe at Gracias
a = 0.25*np.pi*ID**2
v_influent = q/a
KE_influent = v_influent**2/(2*ac.GRAVITY)

print('The velocity exiting the influent pipe is ', v_influent)
print('The kinetic energy exiting the influent pipe is ', KE_influent.to(u.cm))

The velocity exiting the influent pipe is  1.91 meter / second
The kinetic energy exiting the influent pipe is  18.6 centimeter


## Calculate Ke = v^2/2g through the plate settlers at Gracias


In [ ]:
# calculate the kinetic energy through the plate settlers at Gracias

q = (60 * u.L/u.s).to(u.m**3/u.s)
s = 0.0250 * u.m # edge-to-edge distance between plate settlers (ie. does not include plate thickness)
n = 24 # number of plate settlers in entrance tank
w = 0.6 * u.m # plate settler width
area = s*(n+1)*w
v = q/area
Ke = v**2/(2*ac.GRAVITY)

print('The hypothetical velocity through the plate settlers assuming uniform flow is ', v)
print('The kinetic energy through the plate settlers at Gracias is ', Ke.to(u.cm))

The hypothetical velocity through the plate settlers assuming uniform flow is  0.16 meter / second
The kinetic energy through the plate settlers at Gracias is  0.1305 centimeter


## Calculate the velocity through the channel leading to the LFOM

In [ ]:
# calculate the velocity through the channel leading to the LFOM

q = (60 * u.L/u.s).to(u.m**3/u.s)
d = 0.6278 * u.m # depth of water immediately after the plate settlers measured to the top hole on the LFOM (ie. assuming holes are completely covered at 60 L/s flow)
w = 0.6 * u.m # width of ET
area_LFOMchan = d*w
v_LFOMchan = q/area_LFOMchan
Ke_LFOMchan = v_LFOMchan**2/(2*ac.GRAVITY)

print('The velocity in the channel leading to the LFOM is ', v_LFOMchan)
print('The kinetic energy in the channel leading to the LFOM ', Ke_LFOMchan.to(u.cm))

The velocity in the channel leading to the LFOM is  0.1593 meter / second
The kinetic energy in the channel leading to the LFOM  0.1294 centimeter


**Reflection:** It makes sense that neither the velocity or Ke change very much in these two calculations, because the cross-sectional area for flow didn't really change, and v = q/a. The Ke through the plate settlers is well below 5 cm (5 cm would be considered very high)

## Calculate the required length of the entrance tank

In [8]:
# calculate the required ET length for a 60 L/s plant, assuming it has the same width as the flocculators at Gracias

q = (60 * u.L/u.s).to(u.m**3/u.s)
T = 20 * u.degC
nu = ac.viscosity_kinematic_water(T)
rho_H20 = ac.density_water(T)
rho_p = 2650 *u.kg/u.m**3 # density of sand
d_p = (0.1 * u.mm).to(u.m) # minimum grit diameter to be removed according to Kawamura
w = 0.60 * u.m # assume ET is same width as the flocculator
v_c = ((rho_p - rho_H20)*ac.GRAVITY*d_p**2)/(18*nu*rho_H20)
print('The cricitcal velocity is ',v_c.to(u.mm/u.s))

# check that flow is in laminar regime for Stokes' law to be valid
Re = v_c*d_p/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area = q/v_c
length_ET = area/w
print('The entrance tank length is ', length_ET)

The cricitcal velocity is  8.984 millimeter / second
The Reynolds number is  0.8952 dimensionless
The entrance tank length is  11.13 meter


**Reflection:** The calculated critical velocity is 8.984 mm/s, which is very close to the 8 mm/s that Monroe suggested. The Reynolds number is well within the laminar regime so using Stokes' law was an appropriate decision. The entrance tank requires a length of 11.13 meters. For comparison, the length of one treatment train at Gracias (sed tanks + flocculators) is approxiamtely 15 m, and the width of one treatment train is about 7 m. 

# Information and Notes

## Information sources
1.  [Capstone Design: Improved Entrance Tank Design](https://github.com/AguaClara/CEE4520/blob/master/Previous_Final_Projects/2019F/Improved_Entrance_Tank.ipynb)
1.   MWH textbook
1.   [Susumu Kawamura textbook](https://photos.app.goo.gl/6KWgu6m9uSKmRAKH9)
1.   [Sed tank design code example](https://colab.research.google.com/github/AguaClara/CEE4520/blob/master/Unit_Process_Designs/Sed.ipynb)
1. [Gracias CAD Design](https://cad.onshape.com/documents/cb0559d6c3bd85907f8704d3/w/8b31f12d476b14cf91bd29d9/e/6d7584b2804b3c166ba1e991)

## Problems with the current design

1.   Explore options to reduce the kinetic energy of the incoming water. This results in lots of wave action and splashing. Perhaps inject the water deeper into the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface. Or direct the water against the wall of the tank? Perhaps  2" diameter diffusers (so that clogging isn't a problem) in the vertical entrance pipe with the diffusers pointing upstream toward the end wall of the entrance tank? 
1.   Plate settlers don't work because the velocities are too high and flow isn't distributed at all unformly through the plates. Recommend either improving flow distribution (see 4520 capstone design project) or reverting to a simple horizontal flow tank to capture grit. The simple horizontal sed tank should be explored as the simplest option.
1.   Velocity through the bar screen is too high. Increase the area for the bar screen and include a minimum of 2 bar screens and 3 tracks for inserting the screens so that one can be removed and cleaned and replaced. Screen can be angled to enable it to have more surface area.
1.   Velocity of approach for the LFOM is too high causing wave action. See if we can predict the wave action that is observed at Gracias. The waves are simple a conversion of kinetic energy into potential energy, $v=\sqrt{2gh}$. 

The new entrance tank will likely take more horizontal space OR it is likely that it simply is the full depth of the flocculator with several hoppers that drain any accumulated sediment.

## Strategy

1.  Design the ET in python
1.  Draw the ET in Onshape
1.  Transition on the python code to Feature Script
1.  Intergrate the new ET with the flocculator, LFOM, and CDC



## Notes from meeting with Monroe








### Wed. June 24
- clarification: we are concerned with splashing everywhere in the ET
- start with worst case scenario, if the pipe coming into the entrance tank was directed right towards the surface. Take the velocity coming out of influent pipe, and find the velocity flowing uniformly between the plates. There is very little headloss between the plates, so it's difficult to get an even distribution of flow
- From powerpoint about turbulence:
  - b is the width of the jet
  - Vc is the velocity at the center of the jet
  - s equals the distance forward
  - there is a pproximately a 1/10 ratio between s and b, ie. b = 0.116s or something like that (number might not be exactly correct). Suppose the orifices on a trash rack are spaced out a centimeter. This means that 10 cm out, all of the jets will have merged.
- We want to create many small eddies, because they dissipate the fastest, the trash rack can possibly help with this.
  - Make the trash rack taller and potentially at a slope to increase surface area.
  - at the far end of the flocculator, the channel is 2 m deep (we think), upstream add 50 cm headloss, then add 10 cm drop in the LFOM, and then an additional 20 cm for a total of 2.8 m height in the entrance tank. The trash racks would be placed after the first hopper and would go as deep as the top of the hopper. There is a physical limitation to how deep the trash rack can be, because the operators have to be able to lift it to clean it.
- There are going to be 4 sided hoppers, need to determine how many and the length of the entrance tank. The number of drains depends on the size of particles. Confirm capture velocity from Kawamura textbook being 8 mm/s and this will set the length of the sed tank
- the influent pipe would have a firnco coupling so it can be taken out, and possibly capped at the bottom end to prevent water from flowing straight down into the first hopper. This would be bad because it would not promote even flow throughout the tank. Influent pipe could also have a slot on the side instead of diffusers.
- the eddy velocity in a pipe is roughly 10% of the mean velocity

**Action Items:**
- Monroe will send the pictures of the Kawamura textbook **complete**
- Calculate v^2/2g for the plate settlers as Gracias. 5 cm would be a lot. **complete**
- Calculate the velocity in the channel leading to the LFOM (where it gets a lot more shallow really fast) **complete**
- Calculate how long the ET would need to be for a 60 L/s plant when designed as a horizontal sedimentation chamber. Assume the same flocculator width as Gracias. **complete**

**Reflections & Future Questions**
- Will flow expansion around a barrier dissipate more kinetic energy than water flowing through a barrier with a bunch of hole drilled into it? 
- Do non-AguaClara entrance tanks have similar problems? Can we learn anything from how problems have been dealt with in the past? 
- is the energy dissipation rate of water coming out of the influent pipe related to the energy dissipation rate of the turbulent eddies?
- For predicting wave action around LFOM, what observations at Gracias should this be based off of?
- are there any drawbacks to injecting the water deeper in the entrance tank?
- the bottom geometry of the entrance tank should be selected to ensure that maintenance and the removal of grit is simple and effective. 

**Kawamura Textbook Notes:**

- "a hydraulic scale model conducted by the author clearly demonstrated that when the screen is installed properly, it functions as an inlet diffuser wall"
- "The square tank is satisfactory for the purpose of wastewater treatment but does not perform well as a water treatment grit chamber. The main problem with the detritus [square] tank is a function of its physical characteristics. The short tank length and the uneven inflow to the tank cause severe flow short-circuiting. ... To provide good flow characteristics for grit settling, the chamber should be rectangular with a contracted inlet."

## Task: Reduce the kinetic energy of the influent water

**Problem:** The high kinetic energy of the influent water causes lots of wave action and splashing 

**Possible Solutions**
1. inject the water deeper into the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface.
1. direct the water against the wall of the tank
1. use 2" diameter diffusers (so that clogging isn't a problem) in the vertical entrance pipe with the diffusers pointing upstream toward the end wall of the entrance tank



### Task 1 Solution 1: Inject water deeper in the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface

**Edit 6/24: not sure that we really need to any math to determine how deep the influent tube should be. would probably just position it as low as possible. Ask Monroe later.**

- Define "somewhat dissipated."
  - From AC textbook: "Eddy turnover time, $t_{eddy}$, is the time it takes for the eddy to travel a distance equal to its length-scale. Thus the eddy turnover time provides a good estimate of the time required for mixing to occur at the length scale of the eddy. **We assume that the energy of the large eddy is dissipated into smaller length scales in the time $t_{eddy}$:**"

- Which equations govern the dissipation of turbulent eddies?
$$t_{eddy} \approx \frac{L_{eddy}}{v_{eddy}}$$

   The rate of energy loss to smaller scales
$$\bar\varepsilon \approx\frac{v_{eddy}^3}{L_{eddy}}$$

   and therefore...

$$v_{eddy} \approx \left( \bar\varepsilon \, L_{eddy} \right)^\frac{1}{3}$$

- "The largest eddies are limited in size by the smallest dimension normal to the direction of flow"

   *Therefore the length scale of a turbulent eddy in the entrance tank is either the vertical distance between the water surface and the 'baffle wall', or the width of the entrance tank (whichever distance is smaller).*???

- How is water currently being injected?
  - see onshape drawing, there's a tee at the end of the pipe so water flows out horizontally towards the side walls of the entrance tank

- For reference: The form of energy loss from the influent pipe is expansion, (ie. the following equations are relevant)

$$h_e = K\frac{\bar v_{out}^2}{2g}$$

$$\bar\varepsilon = K\frac{\bar v_{out}^3}{2H}$$

$$G_{CS} = \bar v_{out}\sqrt{\frac{K\bar v_{out}}{2H\nu}}$$

 
- Determine the injection depth needed so that turbulent eddies are "somewhat dissipated"
  
